# Texterfassung Steuerzentrale für Yijing

## 1. Imports und Konfiguration

In [2]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any

import google.generativeai as genai
import pandas as pd
from IPython.display import Markdown, display

### API-Konfiguration

In [3]:
# Konfiguration des API-Schlüssels
api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("Die Umgebungsvariable 'API_KEY' ist nicht gesetzt.")
genai.configure(api_key=api_key)

### Modellwahl

In [53]:
# Auswahl des Modells
model_speed = {
    'dumb': 'gemini-1.5-flash-8b',
    'fast': 'gemini-1.5-flash-latest',
    'clever': 'gemini-1.5-pro-latest',
    'experimental': 'gemini-exp-1121',
}
model_type = os.getenv("MODEL_TYPE", model_speed['clever'])
print("Verwendetes Modell:", model_type)

Verwendetes Modell: gemini-1.5-pro-latest


## 2. Datenimport

### Yijing Text einlesen

In [35]:
yijing_txt_path = Path('yijing/resources/yijing.txt')

# Funktion zum Lesen des Yijing-Texts
def read_yijing_txt(path: Path = yijing_txt_path) -> str:
    with path.open('r', encoding='utf-8') as f:
        return f.read()

# Laden der Daten
yijing_txt = read_yijing_txt()
yijing_chapters = yijing_txt.split('\n')
print("Yijing-Text erfolgreich geladen.")
print("Anzahl der Kapitel:", len(yijing_chapters))

Yijing-Text erfolgreich geladen.
Anzahl der Kapitel: 64


### Erste Einblicke in die Daten

In [36]:
# Anzeige der ersten 500 Zeichen des Yijing-Texts
print(yijing_chapters[51])

KUI / DER GEGENSATZ oben Li, das Haftende, das Feuer unten Dui, das Heitere der See Das Zeichen besteht aus dem oberen Urzeichen Li, die Flamme, die nach oben flammt, und dem Urzeichen Dui, der See, unten, der nach unten sickert. Diese Bewegungen stehen zueinander im Gegensatz. Ferner ist Li die zweite und Dui die jüngste Tochter. Obwohl sie im selben Hause wohnen, gehören sie doch verschiedenen Männern an und ihr Wille ist daher nicht gemeinsam, sondern auf Gegensätzliches gerichtet.   Das Urteil Der Gegensatz In kleinen Sachen Heil! Wenn die Menschen in Gegensatz und Entfremdung leben, so läßt sich ein großes gemeinsames Werk nicht ausführen. Die Gesinnungen gehen zu weit auseinander. Vor allem darf man nicht schroff vorgehen, wodurch der Gegensatz nur noch verschärft würde, sondern muß sich auf allmähliche Wirkungen im Kleinen beschränken. Hier ist noch Heil zu erwarten, da die Lage so ist, daß der Gegensatz nicht jede Verständigung ausschließt. Der Gegensatz, der im allgemeinen als

## 3. Datenverarbeitung

### Strukturierter JSON-Text des ersten Kapitels

In [37]:
yijing_processed = """
{
  "hexagram": {
    "name": "GUAI / DER DURCHBRUCH",
    "subtitle": "Die Entschlossenheit",
    "trigrams": {
      "above": {
        "name": "Dui",
        "attributes": "das Heitere, der See"
      },
      "below": {
        "name": "Kien",
        "attributes": "das Schöpferische, der Himmel"
      }
    },
    "meaning": {
      "description": "Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.",
      "season": "dritter Monat (April-Mai)"
    }
  },
  "judgment": {
    "description": "Der Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen. Der Wahrheit gemäß muß sie verkündet werden. Gefahr! Man muß seine eigene Stadt benachrichtigen. Nicht fördernd ist es, zu den Waffen zu greifen. Fördernd ist es, etwas zu unternehmen.",
    "analysis": [
      "Leidenschaft und Vernunft können nicht zusammen bestehen, daher ist ein unbedingter Kampf notwendig.",
      "Entschlossenheit muß auf Stärke und Freundlichkeit beruhen.",
      "Kompromisse mit dem Schlechten sind nicht möglich.",
      "Der Kampf darf nicht direkt durch Gewalt geführt werden.",
      "Der Edle beginnt bei sich selbst, um das Böse zu entwaffnen."
    ]
  },
  "image": {
    "description": "Der See ist an den Himmel emporgestiegen: das Bild des Durchbruchs.",
    "lesson": "Der Edle spendet Reichtum nach unten hin und scheut es, bei seiner Tugend zu verweilen.",
    "warning": "Sammeln führt zu Zerstreuen; rechtzeitige Vorbereitung kann einem gewaltsamen Zusammenbruch vorbeugen."
  },
  "lines": [
    {
      "position": "Anfangs eine",
      "text": "Mächtig in den vorwärtsschreitenden Zehen. Geht man hin und ist der Sache nicht gewachsen, so macht man einen Fehler.",
      "interpretation": "Zu Beginn ist entschlossenes Voranschreiten schwierig. Blindes Draufgängertum führt zu unheilvollen Folgen."
    },
    {
      "position": "Neun auf zweitem Platz",
      "text": "Alarmruf. Abends und nachts Waffen. Fürchte nichts.",
      "interpretation": "Vorsicht und Wachsamkeit schützen vor Gefahren. Besonnenheit ist der rechte Weg zur Sicherheit."
    },
    {
      "position": "Neun auf drittem Platz",
      "text": "Mächtig in den Backenknochen zu sein bringt Unheil.",
      "interpretation": "Entschlossenheit ist notwendig, aber äußere Stärke zur falschen Zeit kann die Lage verschlimmern."
    },
    {
      "position": "Neun auf viertem Platz",
      "text": "An den Oberschenkeln ist keine Haut, und das Gehen fällt schwer.",
      "interpretation": "Eigensinn führt zu Konflikten. Würde man Ratschläge annehmen, könnte alles gutgehen."
    },
    {
      "position": "Neun auf fünftem Platz",
      "text": "Dem Unkraut gegenüber braucht es feste Entschlossenheit.",
      "interpretation": "Hindernisse müssen mit Entschlossenheit überwunden werden, ohne vom Weg abzukommen."
    },
    {
      "position": "Oben eine Sechs",
      "text": "Kein Ruf! Schließlich kommt Unheil.",
      "interpretation": "Nachlässigkeit beim Entfernen des Bösen führt zu erneutem Übel. Gründliche Arbeit ist notwendig."
    }
  ]
}
"""


In [38]:
# Umwandlung des JSON-Textes in ein Python-Dictionary
yijing_data = json.loads(yijing_processed)
print("Erstes Kapitel erfolgreich verarbeitet.")

Erstes Kapitel erfolgreich verarbeitet.


In [39]:
#yijing_data

## 4. Funktionen zur Verarbeitung eines Hexagramms

In [40]:
def process_hexagram(text: str, model: genai.GenerativeModel) -> Dict[str, Any]:
    """
    Verarbeitet einen Hexagramm-Text und gibt strukturierte Daten zurück.
    """
    prompt = HEXAGRAM_PROMPT + text
    result = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=None  # Verwenden Sie None, wenn kein Schema benötigt wird
        ),
    )
    return result.candidates[0].content

## 5. Initialisierung des Modells

### Systemanweisung für das Modell

In [41]:
HEXAGRAM_PROMPT = """
Du bist ein I Ging-Experte mit der Aufgabe, Hexagramm-Texte zu analysieren und in ein spezifisches JSON-Format zu überführen.

Wandle den Eingabetext in folgendes Format um:

{
  "hexagram": {
    "name": "Name des Hexagramms (z.B. 'GUAI / DER DURCHBRUCH')",
    "subtitle": "Untertitel oder Kernbedeutung", (z.B. 'Die Entschlossenheit')
    "trigrams": {
      "above": {
        "name": "Name des oberen Trigramms", (z.B. 'Dui')
        "attributes": "Eigenschaften des Trigramms" (z.B. 'das Heitere, der See')
      },
      "below": {
        "name": "Name des unteren Trigramms", (z.B. 'Kien')
        "attributes": "Eigenschaften des Trigramms" (z.B. 'das Schöpferische, der Himmel')
      }
    },
    "meaning": {
      "description": "Hauptbedeutung des Hexagramms", (z.B. 'Das Zeichen bedeutet...')
      "season": "Zugeordnete Jahreszeit" (z.B. 'dritter Monat (April-Mai)')
    }
  },
  "judgment": {
    "description": "Der Urteilstext", (z.B. 'Der Durchbruch...')
    "analysis": [
      "Liste von Analysepunkten zum Urteil" (z.B. 'Leidenschaft und Vernunft...')
    ]
  },
  "image": {
    "description": "Beschreibung des Bildes", (z.B. 'Der See ist an den Himmel emporgestiegen...')
    "lesson": "Lehre für den:die Edle:n", (z.B. 'Der:die Edle spendet Reichtum...')
    "warning": "Warnung oder zusätzliche Hinweise" # (z.B. 'Sammeln führt zu Zerstreuen...')
  },
  "lines": [
    {
      "position": "Position der Linie", (z.B. 'Anfangs eine Sechs')",
      "text": "Text der Linie", z.B. 'Mächtig in den vorwärtsschreitenden Zehen...'
      "interpretation": "Interpretation der Linie", z.B. 'Zu Beginn ist entschlossenes Voranschreiten...'
    }
  ]
}

Wichtige Hinweise:
1. Extrahiere alle relevanten Informationen aus dem Eingabetext
2. Behalte die originale Formulierung wo möglich bei
3. Stelle sicher, dass alle Pflichtfelder gefüllt sind
4. Das lines-Array muss genau 6 Einträge enthalten
5. Füge fehlende Informationen mit "Keine Information verfügbar" ein

Analysiere nun den folgenden Hexagramm-Text und gib ihn im spezifizierten JSON-Format zurück:
"""

In [42]:
instruction = HEXAGRAM_PROMPT

In [43]:
# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

# Chat-Instanz starten
chat = model.start_chat()
print("Modell und Chat-Instanz erfolgreich initialisiert.")

Modell und Chat-Instanz erfolgreich initialisiert.


## 6. Verarbeitung und Export

### Hexagramm verarbeiten und anzeigen

In [44]:
def process_hexagram_text(hexagram_text: str) -> Dict[str, Any]:
    """
    Verarbeitet einen Hexagramm-Text und gibt strukturierte Daten zurück.

    Args:
        hexagram_text (str): Der Hexagramm-Text.

    Returns:
        dict: Strukturierte Daten des Hexagramms.
    """
    result = chat.send_message(hexagram_text)

    # Überprüfen, ob die Antwort Teile enthält
    if hasattr(result, 'parts') and len(result.parts) > 0:
        json_text = result.parts[0].text
        # Entfernen von Codeblöcken, falls vorhanden
        formatted_text = json_text.strip('```json\n').strip('```')
        
        try:
            # Laden des JSON in ein Python-Dictionary
            data = json.loads(formatted_text)
            display(Markdown("### Verarbeitetes Hexagramm"))
            display(json.dumps(data, indent=4, ensure_ascii=False))
            return data
        except json.JSONDecodeError:
            print("Fehler beim Parsen des JSON.")
    else:
        print("Keine Antwortteile gefunden.")
    
    return {}

In [19]:
# Beispielhafte Nutzung mit dem ersten Kapitel
first_chapter_text = yijing_chapters[0] if 'yijing_chapters' in locals() else yijing_txt[:1000]
extracted_hexagram = process_hexagram_text(first_chapter_text)

### Verarbeitetes Hexagramm

'{\n    "hexagram": {\n        "name": "GUAI / DER DURCHBRUCH",\n        "subtitle": "Die Entschlossenheit",\n        "trigrams": {\n            "above": {\n                "name": "Dui",\n                "attributes": "das Heitere, der See"\n            },\n            "below": {\n                "name": "Kien",\n                "attributes": "das Schöpferische, der Himmel"\n            }\n        },\n        "meaning": {\n            "description": "Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.",\n            "season": "dritter Monat (April-Mai)"\n        }\n    },\n    "judgment": {\n        "description": "Der Durchbruch. Entsc

### Speicherung der Daten als JSON

In [20]:
def save_hexagram_json(hexagram_data: Dict[str, Any], output_dir: Path, filename: str) -> None:
    """
    Speichert die Hexagramm-Daten als JSON-Datei.

    Args:
        hexagram_data (dict): Strukturierte Hexagramm-Daten.
        output_dir (Path): Verzeichnis zum Speichern der JSON-Dateien.
        filename (str): Name der JSON-Datei.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / filename
    with output_file.open('w', encoding='utf-8') as f:
        json.dump(hexagram_data, f, ensure_ascii=False, indent=4)
    print(f"Daten gespeichert unter {output_file}")

In [21]:
# Speichern des extrahierten Hexagramms
if extracted_hexagram:
    save_hexagram_json(extracted_hexagram, Path('export/hexagram_json'), 'hexagram_01.json')

Daten gespeichert unter export/hexagram_json/hexagram_01.json


In [47]:
# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

# Chat-Instanz starten
chat = model.start_chat()
print("Modell und Chat-Instanz erfolgreich initialisiert.")

Modell und Chat-Instanz erfolgreich initialisiert.


In [55]:
i = 18

for chapter in [yijing_chapters[i-1]]:
    # Erstellen des Modells
    model = genai.GenerativeModel(
        model_type,
        system_instruction=instruction
    )

    # Chat-Instanz starten
    chat = model.start_chat()
    print("Modell und Chat-Instanz erfolgreich initialisiert.")

    print(f"Verarbeite Kapitel {i}...")
    extracted_hexagram = process_hexagram_text(chapter)
    print('Kapitel verarbeitet.')
    if extracted_hexagram:
        save_hexagram_json(extracted_hexagram, Path('export/hexagram_json'), f'hexagram_{i:02d}.json')
    i += 1

Modell und Chat-Instanz erfolgreich initialisiert.
Verarbeite Kapitel 18...


### Verarbeitetes Hexagramm

'{\n    "hexagram": {\n        "name": "GUAN / DIE BETRACHTUNG (DER ANBLICK)",\n        "subtitle": "Das Vorbild",\n        "trigrams": {\n            "above": {\n                "name": "Sun",\n                "attributes": "das Sanfte, der Wind"\n            },\n            "below": {\n                "name": "Kun",\n                "attributes": "das Empfangende, die Erde"\n            }\n        },\n        "meaning": {\n            "description": "Das Zeichen kann als das Bild eines Turmes aufgefasst werden, von dem man eine weite Aussicht ringsumher hat und der selbst weithin sichtbar ist. Es symbolisiert einen Herrscher, der das Gesetz des Himmels und die Sitten des Volkes betrachtet und mit seiner guten Regierung ein Vorbild ist.",\n            "season": "achter Monat (September-Oktober)"\n        }\n    },\n    "judgment": {\n        "description": "Die Betrachtung. Die Waschung ist geschehen, aber noch nicht die Darbringung. Vertrauensvoll blicken sie zu ihm auf.",\n        "

Kapitel verarbeitet.
Daten gespeichert unter export/hexagram_json/hexagram_18.json


# XXXXXXXXXXX

In [65]:
# load all json files in export/hexagram_json
# and store them in a list
hexagram_files = list(Path('export/hexagram_json').glob('*.json'))

hexagram_data = []

for file in hexagram_files:
    with file.open('r', encoding='utf-8') as f:
        data = json.load(f)
        hexagram_data.append(data)

hexagram_data


[{'hexagram': {'name': 'SCHONG / DAS EMPORDRINGEN',
   'subtitle': 'Das Empordringen',
   'trigrams': {'above': {'name': 'Kun',
     'attributes': 'das Empfangende, die Erde'},
    'below': {'name': 'Sun', 'attributes': 'das Sanfte, der Wind'}},
   'meaning': {'description': 'Das untere Zeichen, Sun, hat als Bild das Holz, das obere, Kun, bedeutet die Erde. Damit ist der Gedanke verbunden, daß das Holz in der Erde emporwächst. Dieses »Empordringen« ist im Gegensatz zu dem »Fortschritt« (Nr. 35) mit Anstrengung verbunden, wie die Pflanze Kraft braucht, um durch die Erde emporzudringen. Darum steht das Zeichen, obwohl es mit Erfolg verbunden ist, in Beziehung zur Anstrengung des Willens. Der »Fortschritt« zeigt mehr Expansion, das »Empordringen« mehr geradliniges Aufsteigen zu Macht und Einfluß aus Unbekanntheit und Niedrigkeit.',
    'season': 'Keine Information verfügbar'}},
  'judgment': {'description': 'Das Empordringen hat erhabenes Gelingen. Man muß den großen Mann sehen. Fürchte d

In [198]:
df = pd.DataFrame(hexagram_data)
df.head()

,hexagram,judgment,image,lines
0,"{'name': 'SCHONG / DAS EMPORDRINGEN', 'subtitl...",{'description': 'Das Empordringen hat erhabene...,{'description': 'Inmitten der Erde wächst das ...,"[{'position': 'Anfangs eine Sechs', 'text': 'E..."
1,"{'name': 'DSCHUN / DIE ANFANGSSCHWIERIGKEIT', ...",{'description': 'Die Anfangsschwierigkeit wirk...,{'description': 'Wolken und Donner: das Bild d...,"[{'position': 'Anfangs eine Neun', 'text': 'Zö..."
2,"{'name': 'KUI / DER GEGENSATZ', 'subtitle': 'K...",{'description': 'Der Gegensatz. In kleinen Sac...,"{'description': 'Oben das Feuer, unten der See...","[{'position': 'Anfangs eine Neun', 'text': 'Di..."
3,"{'name': 'GUI ME / DAS HEIRATENDE MÄDCHEN', 's...",{'description': 'Das heiratende Mädchen. Unter...,{'description': 'Oberhalb des Sees ist der Don...,"[{'position': 'Anfangs eine Neun', 'text': 'Da..."
4,"{'name': 'I / DIE MUNDWINKEL (DIE ERNÄHRUNG)',...",{'description': 'Die Mundwinkel. Beharrlichkei...,{'description': 'Unten am Berg ist der Donner:...,"[{'position': 'Anfangs eine Neun', 'text': 'Du..."


In [199]:
for col in df.columns:
    display(Markdown(f"### {col}"))
    display(df.loc[0, 'hexagram'])

### hexagram

{'name': 'SCHONG / DAS EMPORDRINGEN',
 'subtitle': 'Das Empordringen',
 'trigrams': {'above': {'name': 'Kun',
   'attributes': 'das Empfangende, die Erde'},
  'below': {'name': 'Sun', 'attributes': 'das Sanfte, der Wind'}},
 'meaning': {'description': 'Das untere Zeichen, Sun, hat als Bild das Holz, das obere, Kun, bedeutet die Erde. Damit ist der Gedanke verbunden, daß das Holz in der Erde emporwächst. Dieses »Empordringen« ist im Gegensatz zu dem »Fortschritt« (Nr. 35) mit Anstrengung verbunden, wie die Pflanze Kraft braucht, um durch die Erde emporzudringen. Darum steht das Zeichen, obwohl es mit Erfolg verbunden ist, in Beziehung zur Anstrengung des Willens. Der »Fortschritt« zeigt mehr Expansion, das »Empordringen« mehr geradliniges Aufsteigen zu Macht und Einfluß aus Unbekanntheit und Niedrigkeit.',
  'season': 'Keine Information verfügbar'}}

### judgment

{'name': 'SCHONG / DAS EMPORDRINGEN',
 'subtitle': 'Das Empordringen',
 'trigrams': {'above': {'name': 'Kun',
   'attributes': 'das Empfangende, die Erde'},
  'below': {'name': 'Sun', 'attributes': 'das Sanfte, der Wind'}},
 'meaning': {'description': 'Das untere Zeichen, Sun, hat als Bild das Holz, das obere, Kun, bedeutet die Erde. Damit ist der Gedanke verbunden, daß das Holz in der Erde emporwächst. Dieses »Empordringen« ist im Gegensatz zu dem »Fortschritt« (Nr. 35) mit Anstrengung verbunden, wie die Pflanze Kraft braucht, um durch die Erde emporzudringen. Darum steht das Zeichen, obwohl es mit Erfolg verbunden ist, in Beziehung zur Anstrengung des Willens. Der »Fortschritt« zeigt mehr Expansion, das »Empordringen« mehr geradliniges Aufsteigen zu Macht und Einfluß aus Unbekanntheit und Niedrigkeit.',
  'season': 'Keine Information verfügbar'}}

### image

{'name': 'SCHONG / DAS EMPORDRINGEN',
 'subtitle': 'Das Empordringen',
 'trigrams': {'above': {'name': 'Kun',
   'attributes': 'das Empfangende, die Erde'},
  'below': {'name': 'Sun', 'attributes': 'das Sanfte, der Wind'}},
 'meaning': {'description': 'Das untere Zeichen, Sun, hat als Bild das Holz, das obere, Kun, bedeutet die Erde. Damit ist der Gedanke verbunden, daß das Holz in der Erde emporwächst. Dieses »Empordringen« ist im Gegensatz zu dem »Fortschritt« (Nr. 35) mit Anstrengung verbunden, wie die Pflanze Kraft braucht, um durch die Erde emporzudringen. Darum steht das Zeichen, obwohl es mit Erfolg verbunden ist, in Beziehung zur Anstrengung des Willens. Der »Fortschritt« zeigt mehr Expansion, das »Empordringen« mehr geradliniges Aufsteigen zu Macht und Einfluß aus Unbekanntheit und Niedrigkeit.',
  'season': 'Keine Information verfügbar'}}

### lines

{'name': 'SCHONG / DAS EMPORDRINGEN',
 'subtitle': 'Das Empordringen',
 'trigrams': {'above': {'name': 'Kun',
   'attributes': 'das Empfangende, die Erde'},
  'below': {'name': 'Sun', 'attributes': 'das Sanfte, der Wind'}},
 'meaning': {'description': 'Das untere Zeichen, Sun, hat als Bild das Holz, das obere, Kun, bedeutet die Erde. Damit ist der Gedanke verbunden, daß das Holz in der Erde emporwächst. Dieses »Empordringen« ist im Gegensatz zu dem »Fortschritt« (Nr. 35) mit Anstrengung verbunden, wie die Pflanze Kraft braucht, um durch die Erde emporzudringen. Darum steht das Zeichen, obwohl es mit Erfolg verbunden ist, in Beziehung zur Anstrengung des Willens. Der »Fortschritt« zeigt mehr Expansion, das »Empordringen« mehr geradliniges Aufsteigen zu Macht und Einfluß aus Unbekanntheit und Niedrigkeit.',
  'season': 'Keine Information verfügbar'}}

In [112]:
df['hexagram']

0     {'name': 'SCHONG / DAS EMPORDRINGEN', 'subtitl...
1     {'name': 'DSCHUN / DIE ANFANGSSCHWIERIGKEIT', ...
2     {'name': 'KUI / DER GEGENSATZ', 'subtitle': 'K...
3     {'name': 'GUI ME / DAS HEIRATENDE MÄDCHEN', 's...
4     {'name': 'I / DIE MUNDWINKEL (DIE ERNÄHRUNG)',...
                            ...                        
59    {'name': 'DUN / DER RÜCKZUG', 'subtitle': 'Der...
60    {'name': 'TSUI / DIE SAMMLUNG', 'subtitle': 'D...
61    {'name': 'SCHI / DAS HEER', 'subtitle': 'Die O...
62    {'name': 'LÜ / DAS AUFTRETEN', 'subtitle': 'Da...
63    {'name': 'HIE / DIE BEFREIUNG', 'subtitle': 'K...
Name: hexagram, Length: 64, dtype: object

In [150]:
# open import/hexagram_linkage.csv in pandas
linkage_df = (pd.read_csv('import/hexagram_linkage.csv', sep=';')
              .replace('ß', 'ss', regex=True)
              .replace('das durchbeissen', 'das durchbeißen', regex=True)
            )
linkage_df['Alter Name (Deutsch)'] = linkage_df['Alter Name (Deutsch)'].str.lower()
linkage_df

,Nummer,Alter Code,Neuer Code,Vollständiger Name (Deutsch),Alter Name (Deutsch)
0,1,KIEN,QIAN,Das Schöpferische (乾),das schöpferische
1,2,KUN,KUN,Das Empfangende (坤),das empfangende
2,3,DSCHUN,ZHUN,Die Anfangsschwierigkeit (屯),die anfangsschwierigkeit
3,4,MONG,MENG,Die Jugendtorheit (蒙),die jugendtorheit
4,5,SÜ,XU,Das Warten (Die Ernährung) (需),das warten (die ernährung)
...,...,...,...,...,...
59,60,DSIE,JIE,Die Beschränkung (節),die beschränkung
60,61,DSCHUNG FU,ZHONG FU,Innere Wahrheit (中孚),innere wahrheit
61,62,SIAU GO,XIAO GUO,Des Kleinen Übergewicht (小過),des kleinen übergewicht
62,63,GI DSI,JI JI,Nach der Vollendung (既濟),nach der vollendung


In [161]:
# open import/hexagramme.csv in pandas
hexagramme_df = pd.read_csv('import/hexagramme.csv')
hexagramme_df

,Nummer,Bezeichnung,Schriftzeichen,Pīnyīn,Line_Values,Title,Hexagram,Description
0,1,Das Schöpferische,乾,qián,111111,Creative Heaven,䷀,"Hexagram 1 is named 乾 (qián), ""Force"". Other v..."
1,2,Das Empfangende,坤,kūn,0,Receptive Earth,䷁,"Hexagram 2 is named 坤 (kūn), ""Field"". Other va..."
2,3,Die Anfangsschwierigkeit,屯,chún,100010,Difficulty at Beginning,䷂,"Hexagram 3 is named 屯 (zhūn), ""Sprouting"". Oth..."
3,4,Die Jugendtorheit,蒙,méng,10001,Youthful Folly,䷃,"Hexagram 4 is named 蒙 (méng), ""Enveloping"". Ot..."
4,5,Das Warten,需,xū,111010,Waiting,䷄,"Hexagram 5 is named 需 (xū), ""Attending"". Other..."
...,...,...,...,...,...,...,...,...
59,60,Die Beschränkung,節,jíe,110010,Limitation,䷻,"Hexagram 60 is named 節 (jié), ""Articulating"". ..."
60,61,Innere Wahrheit,中孚,zhōng fú,110011,Inner Truth,䷼,"Hexagram 61 is named 中孚 (zhōng fú), ""Center Re..."
61,62,Des Kleinen Übergewicht,小過,xiǎo gùo,1100,Small Preponderance,䷽,"Hexagram 62 is named 小過 (xiǎo guò), ""Small Exc..."
62,63,Nach der Vollendung,既濟,jì jì,101010,After Completion,䷾,"Hexagram 63 is named 既濟 (jì jì), ""Already Ford..."


In [200]:
df = pd.DataFrame(hexagram_data)

df_split = (df['hexagram'].apply(pd.Series)
            .replace('haftende', 'haftene', regex=True)
            .replace('das durchbeissen', 'das durchbeißen', regex=True)
            )

df_split['Alter Name (Deutsch)'] = df_split['name'].str.split(' / ').str[1].str.lower()

df_split = (df_split
            .replace('haftende', 'haftene', regex=True)
            .replace('das durchbeißen', 'das durchbeissen', regex=True)
            .replace('das haftene', 'das haftende', regex=True)
            .replace('die wiederkehrt', 'die wiederkehr', regex=True)
            )

df_merged = df_split.merge(linkage_df,
                           left_on='Alter Name (Deutsch)',
                           right_on='Alter Name (Deutsch)',
                           how='outer',
                           indicator=False)

df_merged = df_merged.merge(hexagramme_df, on='Nummer', how='outer', indicator=True)

#df_split = df_split['meaning'].apply(pd.Series)
#df_split = df_split['below'].apply(pd.Series)
#df_merged = df_merged[df_merged['_merge'] != 'both']

#df_merged = df_merged.iloc[:, :4].T

cols = [
    'Nummer',    
    'name',
    #'subtitle',
    #'trigrams',
    #'meaning',
    'Alter Name (Deutsch)',
    'Alter Code',
    'Neuer Code',
    'Vollständiger Name (Deutsch)',
    'Bezeichnung',
    #'Schriftzeichen',
    #'Pīnyīn',
    #'Line_Values',
    #'Title',
    'Hexagram',
    'Description',
    #'_merge'
    ]

#df_merged = df_merged.loc[:, cols].set_index('Nummer')
df_merged#.columns.to_list()
df_merged

,name,subtitle,trigrams,meaning,Alter Name (Deutsch),Nummer,Alter Code,Neuer Code,Vollständiger Name (Deutsch),Bezeichnung,Schriftzeichen,Pīnyīn,Line_Values,Title,Hexagram,Description,_merge
0,KIEN / DAS SCHÖPFERISCHE,Die Kraft,"{'above': {'name': 'Kien', 'attributes': 'das ...",{'description': 'Das Zeichen besteht aus sechs...,das schöpferische,1,KIEN,QIAN,Das Schöpferische (乾),Das Schöpferische,乾,qián,111111,Creative Heaven,䷀,"Hexagram 1 is named 乾 (qián), ""Force"". Other v...",both
1,KUN / DAS EMPFANGENDE,Die Hingebung,"{'above': {'name': 'Kun', 'attributes': 'das E...",{'description': 'Das Zeichen besteht aus laute...,das empfangende,2,KUN,KUN,Das Empfangende (坤),Das Empfangende,坤,kūn,0,Receptive Earth,䷁,"Hexagram 2 is named 坤 (kūn), ""Field"". Other va...",both
2,DSCHUN / DIE ANFANGSSCHWIERIGKEIT,Die Anfangsschwierigkeit,"{'above': {'name': 'Kan', 'attributes': 'das A...","{'description': 'Das Zeichen deutet, wie Himme...",die anfangsschwierigkeit,3,DSCHUN,ZHUN,Die Anfangsschwierigkeit (屯),Die Anfangsschwierigkeit,屯,chún,100010,Difficulty at Beginning,䷂,"Hexagram 3 is named 屯 (zhūn), ""Sprouting"". Oth...",both
3,MONG / DIE JUGENDTORHEIT,Keine Information verfügbar,"{'above': {'name': 'Gen', 'attributes': 'das S...",{'description': 'Auf doppelte Weise ist das Ge...,die jugendtorheit,4,MONG,MENG,Die Jugendtorheit (蒙),Die Jugendtorheit,蒙,méng,10001,Youthful Folly,䷃,"Hexagram 4 is named 蒙 (méng), ""Enveloping"". Ot...",both
4,SÜ / DAS WARTEN (DIE ERNÄHRUNG),Die innere Gewißheit,"{'above': {'name': 'Kan', 'attributes': 'das A...",{'description': 'Alle Wesen bedürfen der Nahru...,das warten (die ernährung),5,SÜ,XU,Das Warten (Die Ernährung) (需),Das Warten,需,xū,111010,Waiting,䷄,"Hexagram 5 is named 需 (xū), ""Attending"". Other...",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,DSIE / DIE BESCHRÄNKUNG,Die Sparsamkeit,"{'above': {'name': 'Kan', 'attributes': 'das A...",{'description': 'Das Zeichen bedeutet die fest...,die beschränkung,60,DSIE,JIE,Die Beschränkung (節),Die Beschränkung,節,jíe,110010,Limitation,䷻,"Hexagram 60 is named 節 (jié), ""Articulating"". ...",both
60,DSCHUNG FU / INNERE WAHRHEIT,Die Aufrichtigkeit,"{'above': {'name': 'Sun', 'attributes': 'das S...",{'description': 'Über dem See weht der Wind un...,innere wahrheit,61,DSCHUNG FU,ZHONG FU,Innere Wahrheit (中孚),Innere Wahrheit,中孚,zhōng fú,110011,Inner Truth,䷼,"Hexagram 61 is named 中孚 (zhōng fú), ""Center Re...",both
61,SIAU GO / DES KLEINEN ÜBERGEWICHT,Keine Information verfügbar,"{'above': {'name': 'Dschen', 'attributes': 'da...",{'description': 'Hier muß notgedrungen das Sch...,des kleinen übergewicht,62,SIAU GO,XIAO GUO,Des Kleinen Übergewicht (小過),Des Kleinen Übergewicht,小過,xiǎo gùo,1100,Small Preponderance,䷽,"Hexagram 62 is named 小過 (xiǎo guò), ""Small Exc...",both
62,GI DSI / NACH DER VOLLENDUNG,Vorsicht am Höhepunkt,"{'above': {'name': 'Kan', 'attributes': 'das A...",{'description': 'Das Zeichen ist die Ausgestal...,nach der vollendung,63,GI DSI,JI JI,Nach der Vollendung (既濟),Nach der Vollendung,既濟,jì jì,101010,After Completion,䷾,"Hexagram 63 is named 既濟 (jì jì), ""Already Ford...",both


In [209]:
df = df_merged.copy()

for col in df.columns:
    print(f"Spalte: {col}")
    print("Inhalt:", df.loc[0, col])
    print()

Spalte: name
Inhalt: KIEN / DAS SCHÖPFERISCHE

Spalte: subtitle
Inhalt: Die Kraft

Spalte: trigrams
Inhalt: {'above': {'name': 'Kien', 'attributes': 'das Schöpferische, der Himmel'}, 'below': {'name': 'Kien', 'attributes': 'das Schöpferische, der Himmel'}}

Spalte: meaning
Inhalt: {'description': 'Das Zeichen besteht aus sechs ungeteilten Strichen. Die ungeteilten Striche entsprechen der lichten, starken, geistigen, tätigen Urkraft. Das Zeichen ist ganz einheitlich stark in seiner Natur. Da ihm keinerlei Schwäche anhaftet, ist es seiner Eigenschaft nach die Kraft. Sein Bild ist der Himmel. Die Kraft wird dargestellt als nicht gebunden an bestimmte räumliche Verhältnisse. Darum wird sie aufgefaßt als Bewegung. Als Grundlage dieser Bewegung kommt die Zeit in Betracht. So ist denn auch die Macht der Zeit und die Macht des Beharrens in der Zeit, die Dauer, in dem Zeichen begriffen. Bei der Erklärung des Zeichens ist durchgehend eine doppelte Deutung zu berücksichtigen: die makrokosmische u

In [210]:
def export_df_to_json(df: pd.DataFrame, output_file: Optional[Path] = None) -> None:
    """
    Converts DataFrame of hexagram information to our standardized JSON schema format.
    
    Args:
        df: DataFrame containing hexagram information
        output_file: Optional path to save JSON file. If None, saves to default location.
    """
    if output_file is None:
        output_file = Path(__file__).parent / 'resources' / 'hexagrams.json'

    hexagrams = {}
    
    for _, row in df.iterrows():
        hexagram_number = int(row['Nummer'])
        
        # Process trigrams data
        trigrams = eval(row['trigrams']) if isinstance(row['trigrams'], str) else row['trigrams']
        
        # Process meaning data
        meaning_data = eval(row['meaning']) if isinstance(row['meaning'], str) else row['meaning']
        
        hexagram_data = {
            "number": hexagram_number,
            "names": {
                "chinese": row['Schriftzeichen'],
                "pinyin": row['Pīnyīn'],
                "german": row['Bezeichnung'],
                "english": row['Title'],
                "character": row['Schriftzeichen'],
                "old_german": row['Alter Name (Deutsch)'],
                "old_code": row['Alter Code'],
                "new_code": row['Neuer Code'],
                "full_german": row['Vollständiger Name (Deutsch)']
            },
            "trigrams": {
                "above": {
                    "name": trigrams['above']['name'],
                    "attributes": trigrams['above']['attributes']
                },
                "below": {
                    "name": trigrams['below']['name'],
                    "attributes": trigrams['below']['attributes']
                }
            },
            "technical": {
                "line_values": row['Line_Values'],
                "unicode_symbol": row['Hexagram']
            },
            "meaning": {
                "description": meaning_data['description'],
                "season": meaning_data.get('season'),
                "subtitle": row['subtitle']
            },
            "descriptions": {
                "main": row['Description'],
                "traditional": meaning_data['description']
            }
        }
        
        hexagrams[hexagram_number] = hexagram_data

    # Ensure the output directory exists
    output_file.parent.mkdir(parents=True, exist_ok=True)

    # Save to JSON file with proper formatting and encoding
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(hexagrams, f, ensure_ascii=False, indent=2)

    print(f"Successfully exported hexagram data to {output_file}")

In [212]:
output_path = Path('export/hexagrams_test_01.json')

df = df_merged.copy()

export_df_to_json(df, output_path)

Successfully exported hexagram data to export/hexagrams_test_01.json


In [194]:
# utils.py

import pandas as pd
from pathlib import Path
from typing import Dict, Optional
import json
from yijing.interpretations import HexagramInterpretation, YijingInterpretations

def load_hexagram_data() -> Dict[int, HexagramInterpretation]:
    """
    Loads and validates hexagram data from JSON storage.
    
    Returns:
        Dict[int, HexagramInterpretation]: Dictionary of validated hexagram interpretations
    
    Raises:
        FileNotFoundError: If hexagram data file is not found
        ValidationError: If data doesn't match expected schema
    """
    resources_dir = Path(__file__).parent / 'resources'
    hexagram_file = resources_dir / 'hexagrams.json'
    
    if not hexagram_file.exists():
        raise FileNotFoundError(f"Hexagram data file not found: {hexagram_file}")
        
    with open(hexagram_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    return {
        int(num): HexagramInterpretation(**hexagram_data)
        for num, hexagram_data in data.items()
    }

def export_df_to_json(df: pd.DataFrame, output_file: Optional[Path] = None) -> None:
    """
    Converts DataFrame of hexagram information to our JSON schema format.
    
    Args:
        df: DataFrame containing hexagram information
        output_file: Optional path to save JSON file. If None, saves to default location.
    """
    if output_file is None:
        output_file = Path(__file__).parent / 'resources' / 'hexagrams.json'
        
    # Transform DataFrame to our schema format
    hexagrams = {}
    for _, row in df.iterrows():
        hexagram_number = int(row['Nummer'])
        
        # Extract trigram information
        trigrams = eval(row['trigrams']) if isinstance(row['trigrams'], str) else row['trigrams']
        
        hexagrams[hexagram_number] = {
            "number": hexagram_number,
            "name_chinese": row['Schriftzeichen'],
            "name_pinyin": row['Pīnyīn'],
            "name_german": row['Vollständiger Name (Deutsch)'],
            "name_wilhelm": row['Alter Code'],
            "name_modern": row['Title'],
            "subtitle": row['subtitle'],
            "binary_sequence": row['Line_Values'],
            "unicode_symbol": row['Hexagram'],
            "upper_trigram": trigrams['above'],
            "lower_trigram": trigrams['below'] if 'below' in trigrams else None,
            "judgement": {
                "wilhelm_text": row['meaning']['description'] if isinstance(row['meaning'], dict) else ""
            }
        }
    
    # Save to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(hexagrams, f, ensure_ascii=False, indent=2)

In [196]:
df = df_merged.copy()  # Your existing DataFrame
export_df_to_json(df, 'export/hexagrams_test.json')

In [187]:
# tools/convert_hexagram_data.py

import json
from pathlib import Path
from typing import Dict

def convert_hexagram_data(input_json: str, output_dir: Path) -> None:
    """
    Converts single JSON file to individual hexagram files.
    
    Args:
        input_json: Path to input JSON file
        output_dir: Directory for output files
    """
    # Create output directories
    hex_dir = output_dir / 'hexagrams'
    hex_dir.mkdir(parents=True, exist_ok=True)
    
    # Load input data
    with open(input_json) as f:
        data = json.load(f)
    
    # Process each hexagram
    binary_index = {}
    for num, hex_data in enumerate(data, start=1):
        # Add binary sequence if not present
        if 'binary_sequence' not in hex_data:
            hex_data['binary_sequence'] = format(num-1, '06b')
        
        # Update index
        binary_index[hex_data['binary_sequence']] = num
        
        # Save individual file
        output_file = hex_dir / f'{num:02d}.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(hex_data, f, indent=2, ensure_ascii=False)
    
    # Save index
    with open(output_dir / 'index.json', 'w') as f:
        json.dump(binary_index, f, indent=2)


In [ ]:

if __name__ == '__main__':
    input_file = 'hexagrams.json'
    output_dir = Path('data')
    convert_hexagram_data(input_file, output_dir)

In [103]:
df_split = df['hexagram'].apply(pd.Series)
    
i = 0

for index, row in df_split.iterrows():
    print(index, row['name'])

0 SCHONG / DAS EMPORDRINGEN
1 DSCHUN / DIE ANFANGSSCHWIERIGKEIT
2 KUI / DER GEGENSATZ
3 GUI ME / DAS HEIRATENDE MÄDCHEN
4 I / DIE MUNDWINKEL (DIE ERNÄHRUNG)
5 SUNG / DER STREIT
6 DSCHEN / DAS ERREGENDE (DAS ERSCHÜTTERNDE, DER DONNER)
7 BI / DIE ANMUT
8 LIN / DIE ANNÄHERUNG
9 HUAN / DIE AUFLÖSUNG
10 HIEN / DIE EINWIRKUNG, DIE WERBUNG
11 BO / DIE ZERSPLITTERUNG
12 FONG / DIE FÜLLE
13 PI / DIE STOCKUNG
14 DA TSCHU / DES GROSSEN ZÄHMUNGSKRAFT
15 I / DIE MEHRUNG
16 GIEN / DAS HEMMNIS
17 SUI / DIE NACHFOLGE
18 DING / DER TIEGEL
19 DA DSCHUANG / DES GROSSEN MACHT
20 GO / DIE UMWÄLZUNG (DIE MAUSERUNG)
21 GIA JEN / DIE SIPPE
22 LI / DAS HAFTENE, DAS FEUER
23 WEI JI / VOR DER VOLLENDUNG
24 DSIEN / DIE ENTWICKLUNG (ALLMÄHLICHER FORTSCHRITT)
25 YÜ / DIE BEGEISTERUNG
26 TAI / DER FRIEDE
27 LÜ / DER WANDERER
28 KIEN / DIE BESCHEIDENHEIT
29 KIEN / DAS SCHÖPFERISCHE
30 SUN / DAS SANFTE
31 SÜ / DAS WARTEN (DIE ERNÄHRUNG)
32 KUN / DAS EMPFANGENDE
33 GEN / DAS STILLEHALTEN, DER BERG
34 DUI / DAS HEITERE,

In [102]:
# open import/hexagramme.csv in pandas
df_hexagramme = pd.read_csv('import/hexagramme.csv')
df_hexagramme.iloc[:, [0,1,-2]].to_markdown()

'|    |   Nummer | Bezeichnung                  | Hexagram   |\n|---:|---------:|:-----------------------------|:-----------|\n|  0 |        1 | Das Schöpferische            | ䷀          |\n|  1 |        2 | Das Empfangende              | ䷁          |\n|  2 |        3 | Die Anfangsschwierigkeit     | ䷂          |\n|  3 |        4 | Die Jugendtorheit            | ䷃          |\n|  4 |        5 | Das Warten                   | ䷄          |\n|  5 |        6 | Der Streit                   | ䷅          |\n|  6 |        7 | Das Heer                     | ䷆          |\n|  7 |        8 | Das Zusammenhalten           | ䷇          |\n|  8 |        9 | Des Kleinen Zähmungskraft    | ䷈          |\n|  9 |       10 | Das Auftreten                | ䷉          |\n| 10 |       11 | Der Friede                   | ䷊          |\n| 11 |       12 | Die Stockung                 | ䷋          |\n| 12 |       13 | Gemeinschaft mit Menschen    | ䷌          |\n| 13 |       14 | Der Besitz von Großem        | ䷍    

### Export der Linien in ein DataFrame

In [22]:
def export_lines_to_dataframe(hexagram_data: Dict[str, Any]) -> pd.DataFrame:
    """
    Exportiert die Linien eines Hexagramms in ein pandas DataFrame.

    Args:
        hexagram_data (dict): Strukturierte Hexagramm-Daten.

    Returns:
        pd.DataFrame: DataFrame mit den Linieninformationen.
    """
    lines = hexagram_data.get('lines', [])
    lines_df = pd.DataFrame(lines)
    return lines_df

# Erstellen des DataFrames
lines_df = export_lines_to_dataframe(extracted_hexagram)
display(lines_df)

,position,text,interpretation
0,Anfangs eine Neun,Mächtig in den vorwärtsschreitenden Zehen. Geh...,"Zu Beginn ist entschlossenes Voranschreiten, f..."
1,Neun auf zweitem Platz,Warnung! Der Ruf ertönt. Am Abend und in der N...,"Es droht Gefahr, aber man soll sich nicht entm..."
2,Neun auf drittem Platz,Mächtig an den Backenknochen bringt es Unheil....,Entschlossenheit kann in Aggressivität umschla...
3,Neun auf viertem Platz,"Am Gesäß gibt's kein Fleisch. Geht man, so wir...",Man ist in einer schwachen Position und stößt ...
4,Neun auf fünftem Platz,"Wie Unkraut geht der Durchbruch. Geht man, so ...",Der Durchbruch geschieht leicht und mühelos. M...
5,Oben eine Sechs,Kein Schreien. Unheil!,"Der Durchbruch ist vollzogen, aber es fehlt an..."


## 7. Zusätzliche Analysen (Optional)

### Beispiel: Analyse der Interpretationen

In [23]:
if not lines_df.empty:
    from collections import Counter
    import re
    
    # Zusammenführen aller Interpretationen
    all_interpretations = ' '.join(lines_df['interpretation'].dropna().tolist())
    
    # Extrahieren von Wörtern
    words = re.findall(r'\w+', all_interpretations.lower())
    word_counts = Counter(words)
    
    # Anzeige der häufigsten Wörter
    common_words = word_counts.most_common(10)
    common_df = pd.DataFrame(common_words, columns=['Wort', 'Häufigkeit'])
    display(Markdown("### Häufigste Wörter in den Interpretationen"))
    display(common_df)

### Häufigste Wörter in den Interpretationen

,Wort,Häufigkeit
0,man,7
1,und,7
2,zu,4
3,aber,4
4,nicht,4
5,kann,4
6,ist,3
7,entschlossenheit,3
8,in,3
9,der,3


# Ende des Notebooks